In [ ]:
!git clone -b 'new_truncation_scheme' 'https://github.com/LuchnikovI/Dynamic-mode-decomposition-for-open-quantum-systems-identification'
%cd 'Dynamic-mode-decomposition-for-open-quantum-systems-identification'
import math
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

from nmd_finite_env import FiniteEnv
from embedding import Embedding
from utils import optimal_K
import pickle

## Helper functions

In [8]:
# Pauli matrices
sigma_x = tf.constant([[0, 1], [1, 0]], dtype=tf.complex128)
sigma_y = 1j * tf.constant([[0, -1], [1, 0]], dtype=tf.complex128)
sigma_z = tf.constant([[1, 0], [0, -1]], dtype=tf.complex128)

# all Pauli matrices in one tensor
pauli = tf.concat([sigma_x[tf.newaxis],
                   sigma_y[tf.newaxis],
                   sigma_z[tf.newaxis]], axis=0)


def data_generation_and_fit(model, rho, eps,
                            total_time=20, time_step=0.3,
                            K=25, auto_K=False):
    '''The function performs the generation of data
    with given parameters and fits them by using DMD.

    Args:
        model: model of non-Markovian dynamics
        rho: complex valued tensor of shape (number_of_lines, 2, 2), initial
            density matrices
        eps: real valued number, std of additive noise
        total_time: real valued number, total time
        time_step: real valued number, time step
        K: integer number, memory depth
        auto_K: boolean number showing if one needs to
            use automatic K determination

    Returns: dict with data, includes the following keys:
        'rank' is the dimension of the Markovian embedding,
        'K' is the memory depth,
        'X_test' is a test sample of dynamics in terms of Bloch vectors
        'X_predicted' is the prediction of test dynamics
        'X_test_noisy' is the noisy version of 'X_test'
        'true_phi_lmbd' is eigenvalues of the quantum channel
        'learned_phi_lmbd' is eigenvalues of the embedding
        'denoised_X' is a denoised trajectory from data set
        'noisy_X' is a noisy trajectory from data set
        'noiseless_X' is a noiseless trajectory from data set
        '''

    # training set without noise
    train_set_wo_noise = model.dynamics(total_time, time_step, rho)
    # noise
    noise = tf.complex(tf.random.normal(train_set_wo_noise.shape, dtype=tf.float64),
                       tf.random.normal(train_set_wo_noise.shape, dtype=tf.float64))
    # training set with noise
    train_set = train_set_wo_noise + eps * tf.complex(tf.random.normal(train_set_wo_noise.shape, dtype=tf.float64),
                                                      tf.random.normal(train_set_wo_noise.shape, dtype=tf.float64))

    # embedding
    emb = Embedding()

    # learn embedding
    if auto_K:
        denoised_t = emb.learn(train_set, eps=eps, auto_K=True, denoise=True)
    else:
        denoised_t = emb.learn(train_set, K=K, eps=eps, denoise=True)
    
    # sample of noisy, noiseless and denoised trajectories from data set
    denoised_X = tf.tensordot(denoised_t[0], pauli, [[1, 2], [2, 1]])
    noisy_X = tf.tensordot(train_set[0], pauli, [[1, 2], [2, 1]])
    noiseless_X = tf.tensordot(train_set_wo_noise[0], pauli, [[1, 2], [2, 1]])
    
    # embedding dim
    rank = emb.rank
    # memory depth
    K = emb.K

    # test set
    rho_test = tf.constant([[1, 0], [0, 0]], dtype=tf.complex128)[tf.newaxis]
    test_set = model.dynamics(total_time, time_step, rho_test)
    # noisy test set
    noise = tf.complex(tf.random.normal(train_set_wo_noise.shape, dtype=tf.float64),
                       tf.random.normal(train_set_wo_noise.shape, dtype=tf.float64))
    test_set_noisy = test_set  + eps * noise
    # prediction
    prediction = emb.predict(test_set_noisy[0, :K],
                             test_set_noisy.shape[1]-K)

    # predicted Bloch vectors
    X_predicted = tf.tensordot(prediction, pauli, [[1, 2], [2, 1]])
    # test Bloch vectors
    X_test = tf.tensordot(test_set, pauli, [[2, 3], [2, 1]])
    # noisy test bloch vectors
    X_test_noisy = tf.tensordot(test_set_noisy, pauli, [[2, 3], [2, 1]])
    #X_pred_input = tf.tensordot(test_set[:, :K], pauli, [[2, 3], [2, 1]])

    # exact channel eigenvalues
    true_gen_lmbd, _ = tf.linalg.eig(model.gen)
    true_phi_lmbd = tf.math.exp(time_step * true_gen_lmbd)

    # learned eigenvalues
    learned_phi_lmbd = emb.channel

    data = {}
    data['rank'] = rank
    data['K'] = K
    data['X_test'] = X_test
    data['X_predicted'] = X_predicted
    data['X_test_noisy'] = X_test_noisy
    data['true_phi_lmbd'] = true_phi_lmbd
    data['learned_phi_lmbd'] = learned_phi_lmbd
    data['denoised_X'] = denoised_X
    data['noisy_X'] = noisy_X
    data['noiseless_X'] = noiseless_X
    return data

def generation_experiments(list_of_mem_dims,
                           list_of_eps,
                           list_of_K,
                           dissipation_ampl=0.1,
                           hamiltonian_ampl=1,
                           time_step=0.2,
                           total_time=40,
                           number_of_lines=4):
    '''Returns data for plotting.

    Args:
        list_of_dims: list with values of environment dimension
        list_of_eps: list with values of noise std
        list_of_K: list with values of memory depth
        dissipation_ampl: real valued number showing the amplitude
            of the dissipator
        hamiltonian_ampl: real valued number showing the amplitude
            of the Hamiltonian part
        time_step: real valued number, time step size
        total_time: real valued number, total simmulation time
        number_of_lines, integer number, number of trajectories

    Returns:
        nested dictionaries with data.'''

    data_dict = {}  
    for mem_dim in list_of_mem_dims:

        #  model for data generation
        model = FiniteEnv(2, mem_dim)
        model.set_rand_gen(dissipation_ampl / ((2 * mem_dim) ** 2 - 1), hamiltonian_ampl)

        # random pure initial states
        psi_re = tf.random.normal((number_of_lines, 2), dtype=tf.float64)
        psi_im = tf.random.normal((number_of_lines, 2), dtype=tf.float64)
        psi = tf.complex(psi_re, psi_im)
        psi = psi / tf.linalg.norm(psi, axis=1, keepdims=True)
        rho = psi[:, tf.newaxis] * tf.math.conj(psi)[..., tf.newaxis]

        #  main loop
        eps_dict = {}
        for eps in list_of_eps:
            K_dict = {}
            for K in list_of_K:
                data = data_generation_and_fit(model,
                                               rho,
                                               eps,
                                               total_time=total_time,
                                               time_step=time_step,
                                               K=K, auto_K=False)
                K_dict[K] = data
            eps_dict[eps] = K_dict
        data_dict[mem_dim] = eps_dict
    return data_dict

## Data generation and saving

In [11]:
tf.random.set_seed(42)

list_of_mem_dims = [2, 3, 4, 5, 6]
list_of_eps = [1e-10, 0.001, 0.003, 0.01, 0.03, 0.1]
list_of_K = [5, 15, 30, 45, 60, 75]
data = generation_experiments(list_of_mem_dims, list_of_eps, list_of_K)
with open('data.pickle', 'wb') as f:
    pickle.dump(data, f)